In [1]:
!wandb login

wandb: Currently logged in as: ttra0058 (use `wandb login --relogin` to force relogin)


In [2]:
import torch
import pandas as pd
import os
import wandb
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread, imread_collection
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
os.environ["WANDB"] = "dryrun"
sweep_config = {
    'name': 'LinearSVC_continue_2',
    'method': 'grid',
    'metric': {
        'name': 'f1-score',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'values': [0.01, 0.1]
        },
        'eta0': {
            'values': [0.0001, 0.001, 0.01, 0.1]
        }
    }
}

In [4]:
sweep_id = wandb.sweep(sweep_config, project="cancer", entity="team17")

Create sweep with ID: eyz1o9kf
Sweep URL: https://wandb.ai/team17/cancer/sweeps/eyz1o9kf


In [5]:
def load_split_train_test(datadir,train_transforms,test_transforms, validation_transforms, validation_proportion=0.10, test_proportion=0.2, bs=2000, random_state=9999):
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    validation_data = datasets.ImageFolder(datadir,
                    transform=validation_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    test_size = int(np.floor(test_proportion * num_train))
    validation_size = int(np.floor(validation_proportion * num_train))
    np.random.seed(random_state)
    np.random.shuffle(indices)
    train_idx, test_idx, validation_idx = indices[test_size+validation_size:], indices[:test_size], indices[test_size:test_size+validation_size]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    validation_sampler = SubsetRandomSampler(validation_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=bs)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=bs)
    validationloader = torch.utils.data.DataLoader(validation_data,
                   sampler=validation_sampler, batch_size=bs)
    return trainloader, testloader, validationloader

In [6]:
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Lambda(lambda x: torch.flatten(x)),
                                      ])
test_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Lambda(lambda x: torch.flatten(x)),
                                      ])
validation_transforms = transforms.Compose([transforms.Resize(224),
                                            transforms.ToTensor(),
                                            transforms.Lambda(lambda x: torch.flatten(x)),
                                           ])

In [7]:
# Data path
path = os.path.join(os.path.dirname(os.getcwd()), 'data/')

In [8]:
trainloader, testloader, validationloader = load_split_train_test(path, train_transforms,test_transforms, validation_transforms)

In [9]:
def train():
    config_defaults = {
        'alpha': 0.001,
        'eta0': 0.001
    }
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    model = SGDClassifier(alpha=config.alpha, learning_rate='constant', verbose=False, eta0=config.eta0)
    
    epochs = 30
    # Config is a variable that holds and saves hyperparameters and inputs
    for epoch in range(epochs):
        # fit the SVM model using SGD optimizer
        for inputs, labels in trainloader:
            model.partial_fit(inputs, labels, classes=np.array([0,1]))
        
        # calculate the F-score and accuracy on the validation sets
        true_positive = 0
        false_negative = 0
        false_positive = 0
        true_negative = 0
        n_validation = 0
        for inputs, labels in validationloader:
            pred = torch.Tensor(model.predict(inputs))
            true_positive += torch.sum(torch.bitwise_and(pred == labels, pred == 0)).item()
            true_negative += torch.sum(torch.bitwise_and(pred == labels, pred == 1)).item()
            false_negative += torch.sum(pred == 1).item() - torch.sum(torch.bitwise_and(pred == labels, pred == 1)).item()
            false_positive += torch.sum(pred == 0).item() - torch.sum(torch.bitwise_and(pred == labels, pred == 0)).item()
            n_validation += labels.shape[0]

        validation_f1_score = true_positive / (true_positive + (false_positive+false_negative)/2)
        validation_accuracy = (true_positive+true_negative)/n_validation

        wandb.log({
            'Epoch': epoch,
            'F1_score': validation_f1_score,
            'Accuracy': validation_accuracy,
                  })

In [10]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: iszew46v with config:
wandb: 	alpha: 0.01
wandb: 	eta0: 0.0001
wandb: Currently logged in as: ttra0058 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,29
F1_score,0.40711
Accuracy,0.52457
_runtime,8961
_timestamp,1628268802
_step,29


Epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
F1_score,▁█▆▁▁▁▃▇▁█▆▄▁▁▂█▁▇▃█▅▁▁█▃▂█▁█▆
Accuracy,█▁▆███▇▅█▁▅▆███▁█▄▇▂▆██▁▇█▁█▁▅
_runtime,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


wandb: Agent Starting Run: d86yas6k with config:
wandb: 	alpha: 0.01
wandb: 	eta0: 0.001
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,29
F1_score,0.0
Accuracy,0.60517
_runtime,8371
_timestamp,1628277178
_step,29


...

Epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
F1_score,█▁▁▁███▁██▁█▁▁██▁▁▁▁▁██▁███▁▁▁
Accuracy,▁███▁▁▁█▁▁█▁██▁▁█████▁▁█▁▁▁███
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇██
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


wandb: Agent Starting Run: mbdc6pp7 with config:
wandb: 	alpha: 0.1
wandb: 	eta0: 0.1
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,29
F1_score,0.56613
Accuracy,0.39483
_runtime,8660
_timestamp,1628327960
_step,29


Epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
F1_score,▁▁██▁▁▁████▁█▁███▁██▁▁██▁██▁██
Accuracy,██▁▁███▁▁▁▁█▁█▁▂▁█▁▁██▁▁█▁▁█▁▁
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
